In [1]:
#import statements

from __future__ import print_function
import sys
import os
import timeit
import phreeqpy.iphreeqc.phreeqc_com as phreeqc_mod

#===================================================================================
#example 11 from PHREEQC in the PHREEQpy examples

def make_initial_conditions():
    """
    Specify initial conditions data blocks.

    Uniform initial conditions are assumed.
    """
    initial_conditions = """
    TITLE Example 11.--Transport and ion exchange.
    SOLUTION 0  CaCl2
        units            mmol/kgw
        temp             25.0
        pH               7.0     charge
        pe               12.5    O2(g)   -0.68
        Ca               0.6
        Cl               1.2
    SOLUTION 1  Initial solution for column
        units            mmol/kgw
        temp             25.0
        pH               7.0     charge
        pe               12.5    O2(g)   -0.68
        Na               1.0
        K                0.2
        N(5)             1.2
        END
    EXCHANGE 1
        equilibrate 1
        X                0.0011
    END
        """
    return initial_conditions


def make_selected_output(components):
    """
    Build SELECTED_OUTPUT data block
    """
    headings = "-headings    cb    H    O    "
    for i in range(len(components)):
        headings += components[i] + "\t"
    selected_output = """
    SELECTED_OUTPUT
        -reset false
    USER_PUNCH
    """
    selected_output += headings + "\n"
    #
    # charge balance, H, and O
    #
    code = '10 w = TOT("water")\n'
    code += '20 PUNCH CHARGE_BALANCE, TOTMOLE("H"), TOTMOLE("O")\n'
    #
    # All other elements
    #
    lino = 30
    for component in components:
        code += '%d PUNCH w*TOT(\"%s\")\n' % (lino, component)
        lino += 10
    selected_output += code
    return selected_output


def initialize(cells, first=False):
    """
    Initialize IPhreeqc module
    """
    phreeqc = phreeqc_mod.IPhreeqc()                 
    phreeqc.load_database(r"C:\Program Files\USGS\IPhreeqcCOM 3.6.2-15100\database\phreeqc.dat")
    initial_conditions = make_initial_conditions()
    phreeqc.run_string(initial_conditions)
    components = phreeqc.get_component_list()
    selected_output = make_selected_output(components)
    phreeqc.run_string(selected_output)
    phc_string = "RUN_CELLS; -cells 0-1\n"
    phreeqc.run_string(phc_string)
    conc = get_selected_output(phreeqc)
    inflow = {}
    initial = {}
    for name in conc:
        if first:
            inflow[name] = conc[name][0]
        else:
            inflow[name] = conc[name][1]
        initial[name] = conc[name][1]
    task = initial_conditions + "\n"
    task += "COPY solution 1 %d-%d\n" % (cells[0], cells[1])
    task += "COPY exchange 1 %d-%d\n" % (cells[0], cells[1])
    task += "END\n"
    task += "RUN_CELLS; -cells %d-%d\n" % (cells[0], cells[1])
    task += selected_output
    phreeqc.run_string(task)
    conc = get_selected_output(phreeqc)
    for name in conc:
        value = [initial[name]] * len(conc[name])
        conc[name] = value
    return phreeqc, inflow, conc


def advect_step(phreeqc, inflow, conc, cells):
    """Advect by shifting concentrations from previous time step.
    """
    all_names = conc.keys()
    names = [name for name in all_names if name not in ('cb', 'H', 'O')]
    for name in conc:
        # shift one cell
        conc[name][1:] = conc[name][:-1]
        conc[name][0] = inflow[name]
    modify = []
    for index, cell in enumerate(range(cells[0], cells[1] + 1)):
        modify.append("SOLUTION_MODIFY %d" % cell)
        modify.append("\t-cb      %e" % conc['cb'][index])
        modify.append("\t-total_h %f" % conc['H'][index])
        modify.append("\t-total_o %f" % conc['O'][index])
        modify.append("\t-totals")
        for name in names:
            modify.append("\t\t%s\t%f" % (name, conc[name][index]))
    modify.append("RUN_CELLS; -cells %d-%d\n" % (cells[0], cells[1]))
    cmd = '\n'.join(modify)
    phreeqc.run_string(cmd)
    conc = get_selected_output(phreeqc)
    return conc


def get_selected_output(phreeqc):
    """Return calculation result as dict.

    Header entries are the keys and the columns
    are the values as lists of numbers.
    """
    output = phreeqc.get_selected_output_array()
    header = output[0]
    conc = {}
    for head in header:
        conc[head] = []
    for row in output[1:]:
        for col, head in enumerate(header):
            conc[head].append(row[col])
    return conc


def run(ncells, shifts, specie_names):
    """Do one run in one process.
    """
    cells = (1, ncells)
    phreeqc, inflow, conc = initialize(cells, first=True)
    outflow = {}
    for name in specie_names:
        outflow[name] = []
    for _counter in range(shifts):
        # advect
        conc = advect_step(phreeqc, inflow, conc, cells)
        for name in specie_names:
            outflow[name].append(conc[name][-1])
    return outflow


def write_outflow(file_name, outflow):
    """Write the outflow values to a file.
    """
    fobj = open(file_name, 'w')
    header = outflow.keys()
    for head in header:
        fobj.write('%20s' % head)
    fobj.write('\n')
    for lineno in range(len(outflow[head])):
        for head in header:
            fobj.write('%20.17f' % outflow[head][lineno])
        fobj.write('\n')


def main(ncells, shifts):
    """Run different versions with and without multiprocessing
    """

    def measure_time(func, *args, **kwargs):
        """Convinience function to measure run times.
        """
        start = timeit.default_timer()
        result = func(*args, **kwargs)
        return result, timeit.default_timer() - start

    print('Dimensions')
    print('==========')
    print('number of cells:   ', ncells)
    print('number of shifts   ', shifts)
    specie_names = ('Ca', 'Cl', 'K', 'N', 'Na')
    outflow, run_time = measure_time(run, ncells, shifts, specie_names)
    if not os.path.exists('PHREEQpy output'):
        os.mkdir('PHREEQpy output')
    write_outflow('PHREEQpy output/example 11.txt', outflow)
    print('run time:', run_time, ' (s)')

if __name__ == '__main__':
    main(ncells=40, shifts=120)


Dimensions
number of cells:    40
number of shifts    120
run time: 2.93082849999999  (s)


In [2]:
#import statements
import phreeqpy.iphreeqc.phreeqc_com as phreeqc_mod
import os

def get_selected_output(phreeqc):
    """Return calculation result as dict.

    Header entries are the keys and the columns
    are the values as lists of numbers.
    """
    output = phreeqc.get_selected_output_array()
    header = output[0]
    conc = {}
    for head in header:
        conc[head] = []
    for row in output[1:]:
        for col, head in enumerate(header):
            conc[head].append(row[col])
    return conc

def run(input_file, first=False):
    """
    Initialize IPhreeqc module
    """
    phreeqc = phreeqc_mod.IPhreeqc()                 
    phreeqc.load_database(r"C:\Program Files\USGS\phreeqc-3.6.2-15100-x64\database\pitzer.dat")
    phreeqc.run_string(input_file)
    conc = get_selected_output(phreeqc)
#     for name in conc:
#         value = [initial[name]] * len(conc[name])
#         conc[name] = value
    return phreeqc, conc

def write_output(file_name, output):
    """Write the outflow values to a file.
    """
    import pandas
    
    fobj = open(file_name, 'w')
    headers = output.keys()
    df = pandas.DataFrame(output, columns = headers)
    display(df)
    fobj.write(df.to_string())
#     for head in headers:
#         fobj.write(str(head))
#     fobj.write('\n')
#     for line_number in range(len(output[head])):
#         for head in headers:
#             fobj.write(str(output[head][line_number]))
#         fobj.write('\n')

def main(input_file):
    """Run different versions with and without multiprocessing
    """
    import timeit

    def measure_time(func, *args, **kwargs):
        """Convenience function to measure run times.
        """
        start = timeit.default_timer()
        phreeqc, conc = func(*args, **kwargs)
        return phreeqc, conc, timeit.default_timer() - start

    phreeqc, conc, run_time = measure_time(run, input_file)
    if not os.path.exists('PHREEQpy output'):
        os.mkdir('PHREEQpy output')
    write_output('PHREEQpy output/test.txt', conc)
    print('run time:', run_time, ' (s)')

#code explorations
input_file = '''TITLE	 test

SOLUTION 0	
temp 	 24.5 	 #average of Al-Taani et al., 2014 and Longinelli and Craig, 1967..
pH 		 8.22 charge #None
pe 		 0.2679 	   #Al-Taani et al., 2014 // 4.00 is the default (?) // 4.00 is the default (?)
units 	 ppm
Mn	0.000306	#Al-Taani et al., 2014 for the Northern Gulf of Aqaba
Fe	0.006281	#Al-Taani et al., 2014 for the Northern Gulf of Aqaba
B	1.344	#Al-Taani et al., 2014
Cl	24756	#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967
Na	16417.2	#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967 describes [Na]=15834
S(6)	9500	#Longinelli and Craig, 1967 and Llyod, 1967
Ca		774	#Abdel-Aal et al., 2015
K		301	#Abdel-Aal et al., 2015
Mg	1646	#Abdel-Aal et al., 2015
Sr		8.3	#Bernat, Church, and Allegre, 1972 from the Mediterranean
Ba	0.011	#Bernat, Church, and Allegre, 1972 from the Mediterranean
Li	0.228	#Stoffyn-Egli and Mackenzie, 1984 for the Mediterranean Sea
-water 	1321367436582.173

SOLUTION 1-12	Initial solution in the RO module
temp 	 25
units 	 ppm
Alkalinity	0
B	0
Ba	0
Br	0
C	0
C(4)	0
Ca	0
Cl	0
E	0
Fe	0
H	0
H(1)	0
K	0
Li	0
Mg	0
Mn	0
Na	0
O	0
O(-2)	0
S	0
S(6)	0
Si	0
Sr	0
Sg	0
-water 	 1321367436582.173

EQUILIBRIUM_PHASES 1-12	
Akermanite		0	0
Anhydrite		0	0
Anthophyllite		0	0
Antigorite		0	0
Aragonite		0	0
Arcanite		0	0
Artinite		0	0
Barite			0	0
Bischofite		0	0
Bloedite		0	0
Brucite		0	0
Burkeite		0	0
Calcite		0	0
Carnallite		0	0
Celestite		0	0
Chalcedony		0	0
Chrysotile		0	0
Diopside		0	0
Dolomite		0	0
Enstatite		0	0
Epsomite		0	0
Forsterite		0	0
Gaylussite		0	0
Glaserite		0	0
Glauberite		0	0
Goergeyite		0	0
Gypsum			0	0
Halite			0	0
Hexahydrite		0	0
Huntite		0	0
Kainite		0	0
Kalicinite		0	0
Kieserite		0	0
Labile_S		0	0
Leonhardite		0	0
Leonite		0	0
Magnesite		0	0
MgCl2_2H2O		0	0
MgCl2_4H2O		0	0
Mirabilite		0	0
Misenite		0	0
Nahcolite		0	0
Natron			0	0
Nesquehonite		0	0
Pentahydrite		0	0
Pirssonite		0	0
Polyhalite		0	0
Portlandite		0	0
Quartz			0	0
Schoenite		0	0
Sepiolite		0	0
Sylvite		0	0
Syngenite		0	0
Talc			0	0
Thenardite		0	0
Trona			0	0
Borax			0	0
Teepleite		0	0


REACTION 0
	H2O -1; 16528092.34989068
REACTION 1
	H2O -1; 16336239.73078653
REACTION 2
	H2O -1; 16144387.11168238
REACTION 3
	H2O -1; 15952534.49257823
REACTION 4
	H2O -1; 15760681.873474082
REACTION 5
	H2O -1; 15568829.254369931
REACTION 6
	H2O -1; 15376976.635265782
REACTION 7
	H2O -1; 15185124.016161634
REACTION 8
	H2O -1; 14993271.397057483
REACTION 9
	H2O -1; 14801418.777953334
REACTION 10
	H2O -1; 14609566.158849185
REACTION 11
	H2O -1; 14417713.539745037
        INCREMENTAL_REACTIONS 	true
#linear permeate

        #Permeate efficiency parameter: 1
        #Head loss parameter: -0.15
    #Effluent module 1:
        #Estimated CF: 1.0000025314235341
        #Estimated solution mass: 1321364091650.0142



SELECTED_OUTPUT
-file			2021-10-05_red_sea_transport_pitzer_scaling_0.txt
-reaction		true
-temperature		true
-totals			 Alkalinity B Ba Br C C(4) Ca Cl E Fe H H(1) K Li Mg Mn Na O O(-2) S S(6) Si Sr Sg
-saturation_indices	 Akermanite Anhydrite Anthophyllite Antigorite Aragonite Arcanite Artinite Barite Bischofite Bloedite Brucite Burkeite Calcite Carnallite Celestite Chalcedony Chrysotile Diopside Dolomite Enstatite Epsomite Forsterite Gaylussite Glaserite Glauberite Goergeyite Gypsum Halite Hexahydrite Huntite Kainite Kalicinite Kieserite Labile_S Leonhardite Leonite Magnesite MgCl2_2H2O MgCl2_4H2O Mirabilite Misenite Nahcolite Natron Nesquehonite Pentahydrite Pirssonite Polyhalite Portlandite Quartz Schoenite Sepiolite Sylvite Syngenite Talc Thenardite Trona Borax Teepleite
-equilibrium_phases	 Akermanite Anhydrite Anthophyllite Antigorite Aragonite Arcanite Artinite Barite Bischofite Bloedite Brucite Burkeite Calcite Carnallite Celestite Chalcedony Chrysotile Diopside Dolomite Enstatite Epsomite Forsterite Gaylussite Glaserite Glauberite Goergeyite Gypsum Halite Hexahydrite Huntite Kainite Kalicinite Kieserite Labile_S Leonhardite Leonite Magnesite MgCl2_2H2O MgCl2_4H2O Mirabilite Misenite Nahcolite Natron Nesquehonite Pentahydrite Pirssonite Polyhalite Portlandite Quartz Schoenite Sepiolite Sylvite Syngenite Talc Thenardite Trona Borax Teepleite
-pH			true
-time			true
-distance		true
-simulation		true
-high_precision	true
-solution
-step
-water

TRANSPORT
-cells			12
-shifts			24
-lengths		0.08466666666666667
-time_step		72	# the Courant condition is satisfied with the cell_length of 0.08466666666666667 m and the feed velocity of 1.1756766269861331e-14 m/s
-initial_time		0
-boundary_conditions	constant	constant 	 # Dirichlet boundary condition
-stagnant		1	1	0.1	0.9 	 # dual domain
#^stagnant cells	^exchange factor	^CP volume	^bulk volume
-punch_cells		1-12
-punch_frequency	12'''
main(input_file)


,sim,state,soln,dist_x,time,step,pH,pe,reaction,temp(C),...,si_Quartz,si_Schoenite,si_Sepiolite,si_Sylvite,si_Syngenite,si_Talc,si_Thenardite,si_Trona,si_Borax,si_Teepleite
0,1,i_soln,0,-99.000000,-99.0,-99,8.177662,0.267900,-9.900000e+01,24.500000,...,-999.999,-6.524759,-999.999,-3.554333,-4.989299,-999.999,-2.465869,-999.999,-12.746078,-7.604786
1,1,i_soln,1,-99.000000,-99.0,-99,7.000000,4.000000,-9.900000e+01,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
2,1,react,0,-99.000000,0.0,1,8.177662,0.267900,1.652809e+07,24.500000,...,-999.999,-6.524759,-999.999,-3.554333,-4.989299,-999.999,-2.465869,-999.999,-12.746077,-7.604786
3,1,transp,1,0.042333,0.0,0,7.000000,4.000000,1.633624e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
4,1,transp,2,0.127000,0.0,0,7.000000,4.000000,1.614439e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
5,1,transp,3,0.211667,0.0,0,7.000000,4.000000,1.595253e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
6,1,transp,4,0.296333,0.0,0,7.000000,4.000000,1.576068e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
7,1,transp,5,0.381000,0.0,0,7.000000,4.000000,1.556883e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
8,1,transp,6,0.465667,0.0,0,7.000000,4.000000,1.537698e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000
9,1,transp,7,0.550333,0.0,0,7.000000,4.000000,1.518512e+07,25.000000,...,-999.999,-999.999000,-999.999,-999.999000,-999.999000,-999.999,-999.999000,-999.999,-999.999000,-999.999000


run time: 2.5469098000000017  (s)


In [7]:
input_file = '''TITLE	 test

SOLUTION 0	
temp 	 24.5 	 #average of Al-Taani et al., 2014 and Longinelli and Craig, 1967..
pH 		 8.22 charge #None
pe 		 0.2679 	   #Al-Taani et al., 2014 // 4.00 is the default (?) // 4.00 is the default (?)
units 	 ppm
Mn	0.000306	#Al-Taani et al., 2014 for the Northern Gulf of Aqaba
Fe	0.006281	#Al-Taani et al., 2014 for the Northern Gulf of Aqaba
B	1.344	#Al-Taani et al., 2014
Cl	24756	#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967
Na	16417.2	#https://www.lenntech.com/composition-seawater.htm in the Red Sea, and Longinelli and Craig, 1967 describes [Na]=15834
S(6)	9500	#Longinelli and Craig, 1967 and Llyod, 1967
Ca		774	#Abdel-Aal et al., 2015
K		301	#Abdel-Aal et al., 2015
Mg	1646	#Abdel-Aal et al., 2015
Sr		8.3	#Bernat, Church, and Allegre, 1972 from the Mediterranean
Ba	0.011	#Bernat, Church, and Allegre, 1972 from the Mediterranean
Li	0.228	#Stoffyn-Egli and Mackenzie, 1984 for the Mediterranean Sea
-water 	1321367436582.173

SOLUTION 1-12	Initial solution in the RO module
temp 	 25
units 	 ppm
Alkalinity	0
B	0
Ba	0
Br	0
C	0
C(4)	0
Ca	0
Cl	0
E	0
Fe	0
H	0
H(1)	0
K	0
Li	0
Mg	0
Mn	0
Na	0
O	0
O(-2)	0
S	0
S(6)	0
Si	0
Sr	0
Sg	0
-water 	 1321367436582.173

EQUILIBRIUM_PHASES 1-12	
Akermanite		0	0
Anhydrite		0	0
Anthophyllite		0	0
Antigorite		0	0
Aragonite		0	0
Arcanite		0	0
Artinite		0	0
Barite			0	0
Bischofite		0	0
Bloedite		0	0
Brucite		0	0
Burkeite		0	0
Calcite		0	0
Carnallite		0	0
Celestite		0	0
Chalcedony		0	0
Chrysotile		0	0
Diopside		0	0
Dolomite		0	0
Enstatite		0	0
Epsomite		0	0
Forsterite		0	0
Gaylussite		0	0
Glaserite		0	0
Glauberite		0	0
Goergeyite		0	0
Gypsum			0	0
Halite			0	0
Hexahydrite		0	0
Huntite		0	0
Kainite		0	0
Kalicinite		0	0
Kieserite		0	0
Labile_S		0	0
Leonhardite		0	0
Leonite		0	0
Magnesite		0	0
MgCl2_2H2O		0	0
MgCl2_4H2O		0	0
Mirabilite		0	0
Misenite		0	0
Nahcolite		0	0
Natron			0	0
Nesquehonite		0	0
Pentahydrite		0	0
Pirssonite		0	0
Polyhalite		0	0
Portlandite		0	0
Quartz			0	0
Schoenite		0	0
Sepiolite		0	0
Sylvite		0	0
Syngenite		0	0
Talc			0	0
Thenardite		0	0
Trona			0	0
Borax			0	0
Teepleite		0	0


REACTION 0
	H2O -1; 16528092.34989068
REACTION 1
	H2O -1; 16336239.73078653
REACTION 2
	H2O -1; 16144387.11168238
REACTION 3
	H2O -1; 15952534.49257823
REACTION 4
	H2O -1; 15760681.873474082
REACTION 5
	H2O -1; 15568829.254369931
REACTION 6
	H2O -1; 15376976.635265782
REACTION 7
	H2O -1; 15185124.016161634
REACTION 8
	H2O -1; 14993271.397057483
REACTION 9
	H2O -1; 14801418.777953334
REACTION 10
	H2O -1; 14609566.158849185
REACTION 11
	H2O -1; 14417713.539745037
        INCREMENTAL_REACTIONS 	true
#linear permeate

        #Permeate efficiency parameter: 1
        #Head loss parameter: -0.15
    #Effluent module 1:
        #Estimated CF: 1.0000025314235341
        #Estimated solution mass: 1321364091650.0142



SELECTED_OUTPUT
-file			2021-10-05_red_sea_transport_pitzer_scaling_0.txt
-reaction		true
-temperature		true
-totals			 Alkalinity B Ba Br C C(4) Ca Cl E Fe H H(1) K Li Mg Mn Na O O(-2) S S(6) Si Sr Sg
-saturation_indices	 Akermanite Anhydrite Anthophyllite Antigorite Aragonite Arcanite Artinite Barite Bischofite Bloedite Brucite Burkeite Calcite Carnallite Celestite Chalcedony Chrysotile Diopside Dolomite Enstatite Epsomite Forsterite Gaylussite Glaserite Glauberite Goergeyite Gypsum Halite Hexahydrite Huntite Kainite Kalicinite Kieserite Labile_S Leonhardite Leonite Magnesite MgCl2_2H2O MgCl2_4H2O Mirabilite Misenite Nahcolite Natron Nesquehonite Pentahydrite Pirssonite Polyhalite Portlandite Quartz Schoenite Sepiolite Sylvite Syngenite Talc Thenardite Trona Borax Teepleite
-equilibrium_phases	 Akermanite Anhydrite Anthophyllite Antigorite Aragonite Arcanite Artinite Barite Bischofite Bloedite Brucite Burkeite Calcite Carnallite Celestite Chalcedony Chrysotile Diopside Dolomite Enstatite Epsomite Forsterite Gaylussite Glaserite Glauberite Goergeyite Gypsum Halite Hexahydrite Huntite Kainite Kalicinite Kieserite Labile_S Leonhardite Leonite Magnesite MgCl2_2H2O MgCl2_4H2O Mirabilite Misenite Nahcolite Natron Nesquehonite Pentahydrite Pirssonite Polyhalite Portlandite Quartz Schoenite Sepiolite Sylvite Syngenite Talc Thenardite Trona Borax Teepleite
-pH			true
-time			true
-distance		true
-simulation		true
-high_precision	true
-solution
-step
-water

TRANSPORT
-cells			12
-shifts			24
-lengths		0.08466666666666667
-time_step		72	# the Courant condition is satisfied with the cell_length of 0.08466666666666667 m and the feed velocity of 1.1756766269861331e-14 m/s
-initial_time		0
-boundary_conditions	constant	constant 	 # Dirichlet boundary condition
-stagnant		1	1	0.1	0.9 	 # dual domain
#^stagnant cells	^exchange factor	^CP volume	^bulk volume
-punch_cells		1-12
-punch_frequency	12'''

def get_selected_output(phreeqc):
    """Return calculation result as dict.

    Header entries are the keys and the columns
    are the values as lists of numbers.
    """
    output = phreeqc.get_selected_output_array()
    header = output[0]
    conc = {}
    for head in header:
        conc[head] = []
    for row in output[1:]:
        for col, head in enumerate(header):
            conc[head].append(row[col])
    return conc

#import statements
import phreeqpy.iphreeqc.phreeqc_com as phreeqc_mod

phreeqc = phreeqc_mod.IPhreeqc()                 
phreeqc.load_database(r"C:\Program Files\USGS\phreeqc-3.6.2-15100-x64\database\pitzer.dat")
phreeqc.run_string(input_file)
conc = get_selected_output(phreeqc)
# initial = {}
# for name in conc:
#     value = [initial[name]] * len(conc[name])
#     conc[name] = value
print(conc)



{'sim': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'state': ['i_soln', 'i_soln', 'react', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp', 'transp'], 'soln': [0, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'dist_x': [-99.0, -99.0, -99.0, 0.042333333333333334, 0.127, 0.21166666666666667, 0.29633333333333334, 0.381, 0.4656666666666667, 0.5503333333333333, 0.635, 0.7196666666666667, 0.8043333333333333, 0.889, 0.9736666666666667, 0.042333333333333334, 0.127, 0.21166666666666667, 0.29633333333333334, 0.381, 0.4656666666666667, 0.5503333333333